<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Wyjanienie-modelu---analiza-bdw/colabs/Similarity_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1935, done.
remote: Counting objects: 100% (548/548), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 1935 (delta 470), reused 411 (delta 364), pack-reused 1387
Receiving objects: 100% (1935/1935), 6.11 MiB | 6.01 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [4]:
!git checkout Wyjanienie-modelu---analiza-bdw

Branch 'Wyjanienie-modelu---analiza-bdw' set up to track remote branch 'Wyjanienie-modelu---analiza-bdw' from 'origin'.
Switched to a new branch 'Wyjanienie-modelu---analiza-bdw'


In [5]:
import pandas as pd
from lib.similarity_utils import *
from lib.const import TRAIN_DATA, VALIDATE_DATA, SIMILARITY_VALUES, RANKINGS

In [6]:
%%time
train_data = pd.read_parquet(TRAIN_DATA, columns=["nconst", "tconst", "category", "primaryName", "knownForTitles",
                                                  "no_nominations_oscars_norm", "no_nominations_globes_norm",
                                                  "no_nominations_emmy_norm", "no_films_norm",
                                                  "average_films_rating_norm", "genres_code", "types_code"])
validate_data = pd.read_parquet(VALIDATE_DATA, columns=["nconst", "tconst", "category", "primaryName", "knownForTitles",
                                                  "no_nominations_oscars_norm", "no_nominations_globes_norm",
                                                  "no_nominations_emmy_norm", "no_films_norm",
                                                  "average_films_rating_norm", "genres_code", "types_code"])

CPU times: user 31.5 s, sys: 7.14 s, total: 38.6 s
Wall time: 54.4 s


# Porównywanie dwóch aktorów

Wyciągnięcie pierwszego aktora

Weź id aktora ze strony imdb. Wpisujemy imię i nazwisko aktora w wyszukiwarkę na imdb i kopiujemy jego id z linku, np. dla Natalie Portman id bierzemy z linku: https://www.imdb.com/name/nm0000204/?ref_=nv_sr_srsg_0

In [7]:
actor1_id = "nm0000204"

In [8]:
actor1_data = get_actor_data(train_data, validate_data, actor1_id)

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
no_nominations_oscars_norm,0.068
no_nominations_globes_norm,0.174
no_nominations_emmy_norm,0.5
no_films_norm,0.006
average_films_rating_norm,0.666


Wyciągnięcie drugiego aktora

In [9]:
actor2_id = "nm0461136"

In [10]:
actor2_data = get_actor_data(train_data, validate_data, actor2_id)

,993366
nconst,nm0461136
tconst,"[tt0368925, tt0366627, tt0409997, tt1307068, t..."
category,actress
primaryName,Keira Knightley
knownForTitles,"[tt0325980, tt2084970, tt0783233, tt0414387]"
no_nominations_oscars_norm,0.045
no_nominations_globes_norm,0.13
no_nominations_emmy_norm,0.5
no_films_norm,0.005
average_films_rating_norm,0.648


Wyniki similarity dla tych dwóch aktorów

In [11]:
similarity_value = similarity_new(actor1_data, actor2_data, True)
print("Wartość similarity dla ", actor1_data[3], " i ", actor2_data[3], " wynosi: ", similarity_value)

Wartość similarity dla  Natalie Portman  i  Keira Knightley  wynosi:  0.20127943573667717


Zapisanie porównywanych par aktorów wraz wartością similarity do pliku na Google Drive.

In [12]:
file_object = open(SIMILARITY_VALUES, 'a')
text = "{}, {}, similarity value: {}\n".format(actor1_data[3], actor2_data[3], similarity_value)
file_object.write(text)
file_object.close()

Zapisanie wniosków, jeśli wyszło coś ciekawego

# Ranking najbardziej podobnych aktorów do podanego

Wyciągnięcie aktora

Weź id aktora ze strony imdb. Wpisujemy imię i nazwisko aktora w wyszukiwarkę na imdb i kopiujemy jego id z linku, np. dla Natalie Portman id bierzemy z linku: https://www.imdb.com/name/nm0000204/?ref_=nv_sr_srsg_0

In [13]:
actor_id = "nm0000204"

In [14]:
actor_data = get_actor_data(train_data, validate_data, actor_id)

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
no_nominations_oscars_norm,0.068
no_nominations_globes_norm,0.174
no_nominations_emmy_norm,0.5
no_films_norm,0.006
average_films_rating_norm,0.666


Ranking 3 najbardziej podobnych aktorów do podanego powyżej

In [15]:
if actor_id in list(train_data['nconst']):
    ids, values = similarity_one_vs_all_new(train_data, actor_data, True)
    ids, values = select_top_similiar(ids, values, 3)
    names = replace_ids_with_names(train_data, ids, 3)
elif actor_id in list(validate_data['nconst']):
    ids, values = similarity_one_vs_all_new(validate_data, actor_data, True)
    ids, values = select_top_similiar(ids, values, 3)
    names = replace_ids_with_names(validate_data, ids, 3)
ranking = print_top_similiar(actor_data[3], names, values)

Najbardziej podobnymi do Natalie Portman aktorami/aktorkami są w kolejności:
  - Penélope Cruz z similarity równym: 0.284
  - Alicia Vikander z similarity równym: 0.283
  - Liv Ullmann z similarity równym: 0.281


Zapisanie do pliku

In [16]:
file_object = open(RANKINGS, 'a')
text = "Najbardziej podobni do {}:\n- {}, similarity: {}\n- {}, similarity: {}\n- {}, similarity: {}\n".format(actor_data[3], names[0], values[0], names[1], values[1], names[2], values[2])
file_object.write(text)
file_object.close()

Zapisanie wniosków, jeśli wyszło coś ciekawego